In [ ]:
import cv2
import numpy as np

image = cv2.imread('./images/targets/img7.jpg')

#image = cv2.resize(image, None, fx=0.5fy=0.5, interpolation=cv2.INTER_LANCZOS4)

imageOrig = image.copy()

bilateral = cv2.bilateralFilter(image,70,75,75)

bilateral = cv2.GaussianBlur(bilateral,(15,15),0)

# median = cv2.medianBlur(image,5)
# cv2.imshow('',median)


kernelSharpen = np.array([[-2,-2,-2],
                          [-2,17,-2],
                          [-2,-2,-2]])

bilateral = cv2.filter2D(bilateral,-1,kernelSharpen)

cv2.imshow('',bilateral)


# cv2.imshow('',bilateral)
cv2.waitKey()

lower = np.array([87, 0, 102])
upper = np.array([179, 255, 255])

hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv, lower, upper)

kernel = np.ones((5,5), np.uint8)

output = cv2.bitwise_and(image,image, mask= mask)


gray = cv2.cvtColor(output,cv2.COLOR_BGR2GRAY)

erosion = cv2.erode(gray,kernel,iterations = 1)
dil = cv2.dilate(erosion,kernel,iterations = 1)


edge = cv2.Canny(dil,20,255)
# cv2.imshow('',edge)
# cv2.waitKey()

# img3 = cv2.cvtColor(output,cv2.COLOR_BGR2GRAY)

contours, hierarchy = cv2.findContours(edge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    (x,y,w,h) = cv2.boundingRect(contour)
    croppedImage = edge[y:y+h,x:x+w] 
    cv2.imshow('',croppedImage)
    cv2.waitKey()

bg = np.zeros(gray.shape,dtype='uint8')

for contour in contours:
    accuracy = 0.007 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, accuracy, True)
    cv2.drawContours(imageOrig, [approx], 0, (0,0,255),2)
    cv2.fillPoly(bg, [approx], (255, 255, 255), 8)

    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
    # Create an empty output image to hold values
    thin = np.zeros(gray.shape,dtype='uint8')

    con, cont = cv2.threshold(bg, 50,255, cv2.THRESH_BINARY)
    cont2 = cont.copy()
    # Loop until erosion leads to an empty set
    while (cv2.countNonZero(cont)!=0):
        # Erosion
        erode = cv2.erode(cont,kernel)
        # Opening on eroded image
        opening = cv2.morphologyEx(erode,cv2.MORPH_OPEN,kernel)
        # Subtract these two
        subset = erode - opening
        # Union of all previous sets
        thin = cv2.bitwise_or(subset,thin)
        # Set the eroded image for next iteration
        cont = erode.copy()

    cv2.imshow('thinned',thin)
    cv2.imshow('',cont2)
    cv2.waitKey()

# # Finds circles in a grayscale image using the Hough transform
# circles = cv2.HoughCircles(dil, cv2.HOUGH_GRADIENT, 1, 100,
#                              param1=100,param2=30,minRadius=0,maxRadius=200)

# # cv2.HoughCircles function has a lot of parameters, so you can find more about it in documentation
# # or you can use cv2.HoughCircles? in jupyter nootebook to get that 
# print(circles)
# # Check to see if there is any detection
# if circles is not None:
#     # If there are some detections, convert radius and x,y(center) coordinates to integer
#     circles = np.round(circles[0, :]).astype("int")

#     for (x, y, r) in circles:
#         # Draw the circle in the output image
#         cv2.circle(imageOrig, (x, y), r, (0,255,0), 3)
#         # Draw a rectangle(center) in the output image
#         cv2.rectangle(imageOrig, (x - 2, y - 2), (x + 2, y + 2), (0,255,0), -1)

# cv2.imshow("Detections",imageOrig)
# cv2.waitKey()

# cv2.imshow('',output)



# # find Harris corners
# gray = np.float32(croppedImage)
# dst = cv2.cornerHarris(gray,2,3,0.04)
# dst = cv2.dilate(dst,None)
# ret, dst = cv2.threshold(dst,0.01*dst.max(),255,0)
# dst = np.uint8(dst)
# # find centroids
# ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
# # define the criteria to stop and refine the corners
# criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
# corners = cv2.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)
# # Now draw them
# res = np.hstack((centroids,corners))
# res = np.int0(res)
# # croppedImage[res[:,1],res[:,0]]=[0,0,255]
# # croppedImage[res[:,3],res[:,2]] = [0,255,0]
# # cv2.imwrite('subpixel5.png',img)
# cv2.imshow('',croppedImage)
# cv2.waitKey()
# cv2.imshow('',gray)


cv2.waitKey()
cv2.destroyAllWindows()

In [10]:
import cv2
import sys
import numpy as np

def nothing(x):
    pass

useCamera=False

# Check if filename is passed
if (len(sys.argv) <= 1) :
    print ("'Usage: python hsvThresholder.py <ImageFilePath>' to ignore camera and use a local image.")
    useCamera = True

# Create a window
cv2.namedWindow('image')

# create trackbars for color change
cv2.createTrackbar('HMin','image',0,179,nothing) # Hue is from 0-179 for Opencv
cv2.createTrackbar('SMin','image',0,255,nothing)
cv2.createTrackbar('VMin','image',0,255,nothing)
cv2.createTrackbar('HMax','image',0,179,nothing)
cv2.createTrackbar('SMax','image',0,255,nothing)
cv2.createTrackbar('VMax','image',0,255,nothing)

# Set default value for MAX HSV trackbars.
cv2.setTrackbarPos('HMax', 'image', 179)
cv2.setTrackbarPos('SMax', 'image', 255)
cv2.setTrackbarPos('VMax', 'image', 255)

# Initialize to check if HSV min/max value changes
hMin = sMin = vMin = hMax = sMax = vMax = 0
phMin = psMin = pvMin = phMax = psMax = pvMax = 0

# Output Image to display
if useCamera:
    cap = cv2.VideoCapture(0)
    # Wait longer to prevent freeze for videos.
    waitTime = 330
else:
    img = cv2.imread('./images/targets/img3.jpg')
    output = img
    waitTime = 33

while(1):

    if useCamera:
        # Capture frame-by-frame
        ret, img = cap.read()
        img = cv2.resize(img,(700,480))
        output = img
    
    # get current positions of all trackbars
    hMin = cv2.getTrackbarPos('HMin','image')
    sMin = cv2.getTrackbarPos('SMin','image')
    vMin = cv2.getTrackbarPos('VMin','image')

    hMax = cv2.getTrackbarPos('HMax','image')
    sMax = cv2.getTrackbarPos('SMax','image')
    vMax = cv2.getTrackbarPos('VMax','image')

    # Set minimum and max HSV values to display
    lower = np.array([hMin, sMin, vMin])
    upper = np.array([hMax, sMax, vMax])

    # Create HSV Image and threshold into a range.
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    output = cv2.bitwise_and(img,img, mask= mask)

    # Print if there is a change in HSV value
    if( (phMin != hMin) | (psMin != sMin) | (pvMin != vMin) | (phMax != hMax) | (psMax != sMax) | (pvMax != vMax) ):
        print("(hMin = %d , sMin = %d, vMin = %d), (hMax = %d , sMax = %d, vMax = %d)" % (hMin , sMin , vMin, hMax, sMax , vMax))
        phMin = hMin
        psMin = sMin
        pvMin = vMin
        phMax = hMax
        psMax = sMax
        pvMax = vMax

    # Display output image
    cv2.imshow('image',output)

    # Wait longer to prevent freeze for videos.
    if cv2.waitKey(waitTime) & 0xFF == ord('q'):
        break

# Release resources
if useCamera:
    cap.release()
cv2.destroyAllWindows()

(hMin = 0 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)
(hMin = -1 , sMin = -1, vMin = -1), (hMax = -1 , sMax = -1, vMax = -1)


In [17]:
import numpy as np
import cv2 as cv
# filename = 'chessboard2.jpg'
# img = cv.imread(filename)
img = cv2.imread('./images/targets/img7.jpg')
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
# find Harris corners
gray = np.float32(gray)
dst = cv.cornerHarris(gray,2,3,0.04)

dst = cv.dilate(dst,None)
ret, dst = cv.threshold(dst,0.01*dst.max(),255,0)
dst = np.uint8(dst)
cv2.imshow('',dst)

cv.waitKey()

# find centroids
ret, labels, stats, centroids = cv.connectedComponentsWithStats(dst)
# define the criteria to stop and refine the corners
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.001)
corners = cv.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)
# Now draw them
res = np.hstack((centroids,corners))
res = np.int0(res)
img[res[:,1],res[:,0]]=[0,0,255]
img[res[:,3],res[:,2]] = [0,255,0]
# cv.imwrite('subpixel5.png',img)
cv2.imshow('',img)

cv.waitKey()
cv.destroyAllWindows()


error: OpenCV(4.3.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
